In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score

In [58]:
class GradientDecentLR:
    def fit(self,X,Y,Debug=0,trials=10,alpha=0.1,early_stop=0):
        
        self.best_params=np.zeros([len(X[0])+1,])
        self.direcs=[True]*(len(X[0])+1)
        
        if Debug==1:
            print("Debug:X.shape:",X.shape)
            print("Debug:Y.shape:",Y.shape)
        for t in range(trials):
            
            if t==0:
                self.cost=0
                for i in range(len(X)):
                    residuals=0
                    residuals+=Y[i]
                    for j in range(len(X[0])+1):
                        if j==0:
                            residuals-= self.best_params[j]
                        else:
                            residuals-= self.best_params[j]*X[i][j-1]
                    
                    self.cost+=residuals**2
                if Debug==1:print("Debug:sum of cost is",self.cost)
                
            #initial direction checking
            if t==0:
                positiveDirecCost=np.zeros([len(X[0])+1,])
                negativeDirecCost=np.zeros([len(X[0])+1,])
                for i in range(len(X[0])+1):
                    for j in range(len(X)):
                        pos_residuals=0
                        neg_residuals=0
                        pos_residuals+=Y[j]
                        neg_residuals+=Y[j]
                        for k in range(len(X[0])+1):
                            if k==i:
                                if k==0:
                                    pos_residuals-= (self.best_params[k]+alpha)
                                    neg_residuals-= (self.best_params[k]-alpha)
                                else:
                                    pos_residuals-= (self.best_params[k]+alpha)*X[j][k-1]
                                    neg_residuals-= (self.best_params[k]-alpha)*X[j][k-1]
                            else:
                                if k==0:
                                    pos_residuals-= self.best_params[k]
                                    neg_residuals-= self.best_params[k]
                                else:
                                    pos_residuals-= self.best_params[k]*X[j][k-1]
                                    neg_residuals-= self.best_params[k]*X[j][k-1]     
                        positiveDirecCost[i]+=pos_residuals**2
                        negativeDirecCost[i]+=neg_residuals**2

                for i in range(len(X[0])+1):
                    if positiveDirecCost[i] <=negativeDirecCost[i]:
                        self.direcs[i]=True
                    if negativeDirecCost[i] < positiveDirecCost[i]:
                        self.direcs[i] =False
                if Debug==1:print(self.direcs)
                
            #parameter updates
            for i in range(len(X[0])+1):
                convergence=0
                proposal_cost=0
                for j in range(len(X)):
                    prop_residuals=0
                    prop_residuals+=Y[j]
                    for k in range(len(X[0])+1):
                        if k==i:
                            if k==0:
                                if self.direcs[i]==True: prop_residuals-= (self.best_params[k]+alpha)
                                if self.direcs[i]==False: prop_residuals-= (self.best_params[k]-alpha)                                        
                            else:
                                if self.direcs[i]==True: prop_residuals-= (self.best_params[k]+alpha)*X[j][k-1]
                                if self.direcs[i]==False: prop_residuals-= (self.best_params[k]-alpha)*X[j][k-1]  
                        else:
                            if k==0:
                                prop_residuals-= self.best_params[k]
                            else:
                                prop_residuals-= self.best_params[k]*X[j][k-1]   
                    proposal_cost+=prop_residuals**2
                if Debug==1:print("Debug:Proposal Cost is ",proposal_cost)
                if self.cost > proposal_cost:
                    if self.direcs[i]==True: self.best_params[i]=self.best_params[i]+alpha
                    if self.direcs[i]==False: self.best_params[i]=self.best_params[i]-alpha
                    self.cost=proposal_cost
                    convergence+=1
            if convergence==0 and early_stop==1:
                print("early stop")
                break
            else:
                continue
            break
        print("Best parameters:",self.best_params)
        print("Min Cost:",self.cost/len(X))
        return self.best_params
    
    def predict(self,X):
        output=np.zeros(len(X))
        for i in range(len(X)):
            for j in range(len(X[0])+1):
                if j==0:
                    output[i]+=self.best_params[j]
                else:
                    output[i]+=self.best_params[j]*X[i][j-1]
        return output

In [59]:
x,y,coef=make_regression(random_state=42, n_samples=100, n_features=5,noise=20.0,bias=-0.0,coef=True)

In [60]:
reg=GradientDecentLR()
reg.fit(x,y,Debug=0,alpha=0.5,trials=200,early_stop=0)

Best parameters: [ 8.5 63.  98.  58.5 53.5 39.5]
Min Cost: 431.34891935529834


array([ 8.5, 63. , 98. , 58.5, 53.5, 39.5])

In [61]:
r2_score(y,reg.predict(x))

0.9779433108062615